In [1]:
pip install pandas scikit-learn tensorflow


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score

df = pd.read_csv("Training.csv")

X = df.drop("prognosis", axis=1)
y = df["prognosis"]

le = LabelEncoder()
Y = le.fit_transform(y)
Y_cat = to_categorical(Y)

X_train, X_test, y_train, y_test = train_test_split(X, Y_cat, test_size=0.3, random_state=20)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

model_ann = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(Y_cat.shape[1], activation='softmax')
])

model_ann.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model_ann.fit(X_train_scaled, y_train, epochs=30, batch_size=32, verbose=1, validation_split=0.2)

y_pred_ann = model_ann.predict(X_test_scaled)
y_pred_classes = y_pred_ann.argmax(axis=1)
y_true_classes = y_test.argmax(axis=1)
print("Accuracy:", accuracy_score(y_true_classes, y_pred_classes))


Epoch 1/30


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


87/87 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.2839 - loss: 2.9798 - val_accuracy: 0.9956 - val_loss: 0.3655
Epoch 2/30
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9340 - loss: 0.4756 - val_accuracy: 1.0000 - val_loss: 0.0221
Epoch 3/30
87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9838 - loss: 0.1412 - val_accuracy: 1.0000 - val_loss: 0.0076
Epoch 4/30
87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9844 - loss: 0.0787 - val_accuracy: 1.0000 - val_loss: 0.0033
Epoch 5/30
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9944 - loss: 0.0505 - val_accuracy: 1.0000 - val_loss: 0.0014
Epoch 6/30
87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9938 - loss: 0.0432 - val_accuracy: 1.0000 - val_loss: 8.1125e-04
Epoch 7/30
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9943 - loss: 0.0352 - val_accuracy: 1.0000 - val_loss: 4.3188e-04
Epoch 8/30
87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9944 - loss: 0.0293 - val_accuracy: 1.0000 - val_loss: 5.

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Reshape

# Reshape input: (samples, timesteps, features)
X_train_lstm = X_train_scaled.reshape((X_train_scaled.shape[0], X_train_scaled.shape[1], 1))
X_test_lstm = X_test_scaled.reshape((X_test_scaled.shape[0], X_test_scaled.shape[1], 1))

model_lstm = Sequential([
    LSTM(128, input_shape=(X_train_lstm.shape[1], 1), return_sequences=False),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(Y_cat.shape[1], activation='softmax')
])

model_lstm.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model_lstm.fit(X_train_lstm, y_train, epochs=5, batch_size=32, verbose=1, validation_split=0.2)

y_pred_lstm = model_lstm.predict(X_test_lstm)
y_pred_classes_lstm = y_pred_lstm.argmax(axis=1)
print("LSTM Accuracy:", accuracy_score(y_test.argmax(axis=1), y_pred_classes_lstm))


Epoch 1/5
87/87 ━━━━━━━━━━━━━━━━━━━━ 17s 169ms/step - accuracy: 0.0679 - loss: 3.5126 - val_accuracy: 0.3062 - val_loss: 2.5596
Epoch 2/5
87/87 ━━━━━━━━━━━━━━━━━━━━ 20s 164ms/step - accuracy: 0.2344 - loss: 2.4769 - val_accuracy: 0.4238 - val_loss: 1.8803
Epoch 3/5
87/87 ━━━━━━━━━━━━━━━━━━━━ 21s 165ms/step - accuracy: 0.3125 - loss: 2.1011 - val_accuracy: 0.3774 - val_loss: 2.1287
Epoch 4/5
87/87 ━━━━━━━━━━━━━━━━━━━━ 20s 166ms/step - accuracy: 0.2507 - loss: 2.6149 - val_accuracy: 0.2192 - val_loss: 2.6240
Epoch 5/5
87/87 ━━━━━━━━━━━━━━━━━━━━ 20s 163ms/step - accuracy: 0.2195 - loss: 2.6452 - val_accuracy: 0.2758 - val_loss: 2.3446
47/47 ━━━━━━━━━━━━━━━━━━━━ 3s 55ms/step
LSTM Accuracy: 0.29132791327913277


In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout

# Reshape input to (samples, timesteps, features)
X_train_cnn = X_train_scaled.reshape((X_train_scaled.shape[0], X_train_scaled.shape[1], 1))
X_test_cnn = X_test_scaled.reshape((X_test_scaled.shape[0], X_test_scaled.shape[1], 1))

model_cnn = Sequential([
    Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train_cnn.shape[1], 1)),
    MaxPooling1D(pool_size=2),
    Dropout(0.3),
    Conv1D(filters=128, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Dropout(0.3),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(Y_cat.shape[1], activation='softmax')
])

model_cnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model_cnn.fit(X_train_cnn, y_train, epochs=5, batch_size=32, verbose=1, validation_split=0.2)

y_pred_cnn = model_cnn.predict(X_test_cnn)
y_pred_classes_cnn = y_pred_cnn.argmax(axis=1)
print("CNN Accuracy:", accuracy_score(y_test.argmax(axis=1), y_pred_classes_cnn))


Epoch 1/5
87/87 ━━━━━━━━━━━━━━━━━━━━ 5s 36ms/step - accuracy: 0.5416 - loss: 1.8987 - val_accuracy: 0.9985 - val_loss: 0.0100
Epoch 2/5
87/87 ━━━━━━━━━━━━━━━━━━━━ 4s 22ms/step - accuracy: 0.9881 - loss: 0.0530 - val_accuracy: 1.0000 - val_loss: 0.0019
Epoch 3/5
87/87 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - accuracy: 0.9925 - loss: 0.0252 - val_accuracy: 1.0000 - val_loss: 5.1247e-04
Epoch 4/5
87/87 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - accuracy: 0.9962 - loss: 0.0140 - val_accuracy: 1.0000 - val_loss: 7.3306e-04
Epoch 5/5
87/87 ━━━━━━━━━━━━━━━━━━━━ 3s 23ms/step - accuracy: 0.9953 - loss: 0.0149 - val_accuracy: 1.0000 - val_loss: 2.2368e-04
47/47 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step
CNN Accuracy: 1.0


In [8]:
from sklearn.feature_selection import SelectKBest, f_classif
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Feature selection (top 100 features)
selector = SelectKBest(score_func=f_classif, k=100)
X_new = selector.fit_transform(X, Y)

# Re-split and scale
X_train, X_test, y_train, y_test = train_test_split(X_new, Y_cat, test_size=0.3, random_state=20)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)

# Optimized ANN
model_opt_ann = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(Y_cat.shape[1], activation='softmax')
])

model_opt_ann.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model_opt_ann.fit(X_train_scaled, y_train, epochs=5, batch_size=32, verbose=1, validation_split=0.2,
                  callbacks=[early_stop, reduce_lr])

# Evaluation
y_pred_opt = model_opt_ann.predict(X_test_scaled)
y_pred_classes = y_pred_opt.argmax(axis=1)
print("Optimized ANN Accuracy:", accuracy_score(y_test.argmax(axis=1), y_pred_classes))


Epoch 1/5


/usr/local/lib/python3.11/dist-packages/sklearn/feature_selection/_univariate_selection.py:111: UserWarning: Features [ 36  45  51  52  53  54  55  71  72  73  77  88 101 104 105 108 111 112
 113 114 118 120] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/usr/local/lib/python3.11/dist-packages/sklearn/feature_selection/_univariate_selection.py:112: RuntimeWarning: divide by zero encountered in divide
  f = msb / msw
/usr/local/lib/python3.11/dist-packages/sklearn/feature_selection/_univariate_selection.py:112: RuntimeWarning: invalid value encountered in divide
  f = msb / msw
/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


87/87 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.2495 - loss: 3.0834 - val_accuracy: 0.8897 - val_loss: 0.9311 - learning_rate: 0.0010
Epoch 2/5
87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8302 - loss: 0.9250 - val_accuracy: 0.9318 - val_loss: 0.2415 - learning_rate: 0.0010
Epoch 3/5
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9114 - loss: 0.3647 - val_accuracy: 0.9478 - val_loss: 0.1448 - learning_rate: 0.0010
Epoch 4/5
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9173 - loss: 0.2836 - val_accuracy: 0.9478 - val_loss: 0.1212 - learning_rate: 0.0010
Epoch 5/5
87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9303 - loss: 0.2161 - val_accuracy: 0.9376 - val_loss: 0.1195 - learning_rate: 0.0010
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Optimized ANN Accuracy: 0.9552845528455285
